In [4]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt
import numpy as np
import keras

In [2]:
!unzip /content/Dataset.zip

Archive:  /content/Dataset.zip
   creating: Dataset/EarlyBlight/
  inflating: Dataset/EarlyBlight/EarlyBlight_1.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_10.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_100.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1000.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1001.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1002.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1003.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1004.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1005.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1006.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1007.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1008.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1009.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_101.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1010.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1011.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_data = keras.preprocessing.image_dataset_from_directory(
    '/content/Dataset',
    batch_size = 70,
    image_size =(240,240),

    shuffle = True,
    seed =123,
    subset ='training',
    validation_split=0.15,
    )
validation_data =keras.preprocessing.image_dataset_from_directory(
    '/content/Dataset',
    batch_size = 70,
    image_size =(240,240),

    shuffle = True,
    seed =123,
    validation_split =0.15,
    subset ='validation',
    )


Found 4072 files belonging to 3 classes.
Using 3462 files for training.
Found 4072 files belonging to 3 classes.
Using 610 files for validation.


In [18]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization

In [19]:
resnet_model = Sequential()
pretrained_model= ResNet50(include_top=False,
                   input_shape=(240,240,3),
                   pooling='avg',
                   weights='imagenet')
for layer in pretrained_model.layers:
  layer.trainable=False
resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(BatchNormalization())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(BatchNormalization())
resnet_model.add(Dense(4, activation='softmax'))

In [21]:
resnet_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [26]:
history = resnet_model.fit(training_data,
                    steps_per_epoch=len(training_data),
                    epochs=20,
                    validation_data=validation_data,
                    validation_steps=len(validation_data))

Epoch 1/20
50/50 [==============================] - 35s 397ms/step - loss: 0.3276 - accuracy: 0.9087 - val_loss: 0.2333 - val_accuracy: 0.9311
Epoch 2/20
50/50 [==============================] - 17s 335ms/step - loss: 0.0650 - accuracy: 0.9786 - val_loss: 0.1327 - val_accuracy: 0.9525
Epoch 3/20
50/50 [==============================] - 17s 335ms/step - loss: 0.0278 - accuracy: 0.9893 - val_loss: 0.1216 - val_accuracy: 0.9492
Epoch 4/20
50/50 [==============================] - 17s 333ms/step - loss: 0.0214 - accuracy: 0.9908 - val_loss: 0.1086 - val_accuracy: 0.9557
Epoch 5/20
50/50 [==============================] - 16s 316ms/step - loss: 0.0164 - accuracy: 0.9951 - val_loss: 0.1015 - val_accuracy: 0.9541
Epoch 6/20
50/50 [==============================] - 19s 368ms/step - loss: 0.0077 - accuracy: 0.9980 - val_loss: 0.1032 - val_accuracy: 0.9508
Epoch 7/20
50/50 [==============================] - 17s 323ms/step - loss: 0.0111 - accuracy: 0.9983 - val_loss: 0.0978 - val_accuracy: 0.9623

In [29]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img = image.load_img('/content/Dataset/EarlyBlight/EarlyBlight_1.jpg', target_size=(240, 240))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension

# Make predictions
predictions = resnet_model.predict(img_array)  # Assuming 'model' is your trained VGG19 model

# Get the class with the highest probability
predicted_class_index = np.argmax(predictions)

# Define your class labels
class_labels = ['class1', 'class2', 'class3']  # Update with your actual class labels

# Get the predicted class label
predicted_class = class_labels[predicted_class_index]

print("Predicted class:", predicted_class)

1/1 [==============================] - 1s 1s/step
Predicted class: class1


In [30]:
import pickle

# Save the trained model to a file using pickle
with open('model.pkl', 'wb') as model_file:
    pickle.dump(resnet_model, model_file)